In [2]:
import os
import cv2
import numpy as np
from court_stitch import load_frames, create_temporal_mask

video_dir = "../input_videos/video_1.mp4"
base_image_dir = "imgs/manual_warp.jpg"
frames = load_frames(video_dir)

detector = cv2.SIFT_create()
bf = cv2.BFMatcher(cv2.NORM_L2)#, crossCheck=True)

cv2.namedWindow('Feature Matches', cv2.WINDOW_NORMAL)
cv2.namedWindow('Warped Image', cv2.WINDOW_NORMAL)

base_img = cv2.imread(base_image_dir)

mask = create_temporal_mask(frames)

valid_mask = (1 - mask.astype(np.uint8))

for frame in frames:
    if frame is None:
        continue

    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.GaussianBlur(frame, (3,3), 3 )
    
    kp1, des1 = detector.detectAndCompute(base_img, None)
    kp2, des2 = detector.detectAndCompute(frame, valid_mask)

    matches = bf.knnMatch(des1, des2, k=2)

    # ratio test
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)

    good = sorted(good, key=lambda x: x.distance)

    img_matches = cv2.drawMatches(base_img, kp1, frame, kp2, good[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    if len(good) > 10:
            src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

            H, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
            img2_warped = cv2.warpPerspective(frame, H, (base_img.shape[1], base_img.shape[0]))

            cv2.imshow('Warped Image', img2_warped)

    cv2.imshow('Feature Matches', img_matches)
    key = cv2.waitKey(0)
cv2.destroyAllWindows()



[INFO] Loaded 12 frames.


In [3]:
import os
import cv2
import torch
import numpy as np
import pickle as pkl
from court_stitch import load_frames, create_temporal_mask, mask_from_track

from lightglue import SuperPoint, LightGlue

device = 'cuda' if torch.cuda.is_available() else 'cpu'

video_dir = "../input_videos/video_1.mp4"
base_image_dir = "imgs/full_court_warped.jpg"
player_track_f = "../stubs/player_track_stubs.pkl"
with open(player_track_f, "rb") as f:
    player_track = pkl.load(f)
frames = load_frames(video_dir, sample_rate=1)

extractor = SuperPoint(pretrained=True).eval().to(device)
matcher = LightGlue(pretrained='superpoint').eval().to(device)

cv2.namedWindow('Warped Image', cv2.WINDOW_NORMAL)

base_img = cv2.imread(base_image_dir)
gray_base = cv2.cvtColor(base_img, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0

# mask = create_temporal_mask(frames)
# masks = [mask_from_track(track, frame) for track, frame in zip(player_track, frames)]
    
# temporal_mask = create_temporal_mask(frames, var_thresh=12)

# masks = [temporal_mask | mask for mask in masks]

# valid_mask = (1 - mask.astype(np.uint8))

for i, frame in enumerate(frames):
    if frame is None:
        continue

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0

    torch_base = torch.from_numpy(gray_base)[None, None].to(device)
    torch_frame = torch.from_numpy(gray_frame)[None, None].to(device)

    feats0 = extractor.extract(torch_base)
    feats1 = extractor.extract(torch_frame)
    matches = matcher({'image0': feats0, 'image1': feats1})

    m0 = matches['matches0'][0].cpu().numpy()
    valid = m0 > -1

    mkpts0 = feats0['keypoints'][0][valid].cpu().numpy()
    mkpts1 = feats1['keypoints'][0][m0[valid]].cpu().numpy()

    # --- Estimate homography ---
    if len(mkpts0) >= 4:
        H, mask = cv2.findHomography(mkpts1, mkpts0, cv2.RANSAC, 200)
        img2_warped = cv2.warpPerspective(frame, H, (base_img.shape[1], base_img.shape[0]))
        cv2.imshow('Warped Image', img2_warped)

    key = cv2.waitKey(0)
cv2.destroyAllWindows()

[INFO] Loaded 117 frames.
